# Fine-Tunning  T5 - SMALL


In [ ]:
!pip install transformers datasets torch

In [ ]:
!pip install accelerate -U

In [ ]:
from datasets import load_dataset
from transformers import pipeline
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, TrainingArguments, Trainer
from transformers import T5Tokenizer, T5ForConditionalGeneration


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon May  6 20:51:33 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   45C    P8              12W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 67.4 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
path_to_train_file = '/content/drive/MyDrive/Colab Notebooks/Tesis Maestria /Spliit Archivos/cultivos_train_FineTuning_OpenaAi.jsonl'
path_to_valid_file = '/content/drive/MyDrive/Colab Notebooks/Tesis Maestria /Spliit Archivos/cultivos_test_FineTuning_OpenaAi.jsonl'

train_dataset = load_dataset('json', data_files={'train': path_to_train_file}, split='train')
valid_dataset = load_dataset('json', data_files={'train': path_to_valid_file}, split='train')


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
model_checkpoint = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_checkpoint)

def extract_qa_pairs(examples):
    questions = []
    answers = []
    for example in examples['messages']:
        if len(example) > 2:
            questions.append(example[1]['content'])
            answers.append(example[2]['content'])
    return {'question': questions, 'answer': answers}

# Extraer y tokenizar los conjuntos de entrenamiento y validación
qa_train_dataset = train_dataset.map(extract_qa_pairs, batched=True)
qa_valid_dataset = valid_dataset.map(extract_qa_pairs, batched=True)

def tokenize_function(examples):
    # Tokenizar tanto las preguntas como las respuestas
    model_inputs = tokenizer(examples['question'], padding="max_length", truncation=True, max_length=128)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['answer'], padding="max_length", truncation=True, max_length=128)
    model_inputs['labels'] = labels['input_ids']
    return model_inputs


# Tokenizar preguntas y respuestas
tokenized_train_datasets = qa_train_dataset.map(tokenize_function, batched=True)
tokenized_valid_datasets = qa_valid_dataset.map(tokenize_function, batched=True)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/2195 [00:00<?, ? examples/s]

Map:   0%|          | 0/549 [00:00<?, ? examples/s]

Map:   0%|          | 0/2195 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3921: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/549 [00:00<?, ? examples/s]

In [ ]:
model = T5ForConditionalGeneration.from_pretrained(model_checkpoint)

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=4,
    per_device_train_batch_size=4,
    learning_rate=5e-6,  # Ajusta según el learning rate base que desees utilizar
    seed=929662638,
    logging_dir='./logs',  # Guarda logs aquí
    logging_steps=10,  # Ajusta este valor según necesites para obtener logs más frecuentes
    weight_decay=0.01,
    save_strategy="steps",  # Guarda el modelo al final de cada época
    evaluation_strategy="steps",  # Evalúa cada cierto número de pasos
    eval_steps=5,  # Ajusta según la frecuencia con la que desees evaluar
    load_best_model_at_end=True,  # Carga el mejor modelo encontrado al final del entrenamiento
    metric_for_best_model="loss",  # Utiliza la pérdida como métrica para determinar el mejor modelo
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_datasets,
    eval_dataset=tokenized_valid_datasets,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
trainer.train()


Step,Training Loss,Validation Loss
5,No log,5.278976
10,5.562500,5.164184
15,5.562500,5.044266
20,5.278700,4.920315
25,5.278700,4.789366
30,5.016100,4.661170
35,5.016100,4.538347
40,4.936000,4.426180
45,4.936000,4.326914
50,4.773300,4.235341


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=2196, training_loss=2.780064645360728, metrics={'train_runtime': 870.6613, 'train_samples_per_second': 10.084, 'train_steps_per_second': 2.522, 'total_flos': 297075254231040.0, 'train_loss': 2.780064645360728, 'epoch': 4.0})

In [ ]:
trainer.save_model("/content/drive/MyDrive/Colab Notebooks/Tesis Maestria /modelo_entrenado_T5.jsonl")

In [ ]:
import os

output_dir = "/content/drive/MyDrive/Colab Notebooks/Tesis Maestria /Modelos Entrenados/modelo_entrenado_T5"
os.makedirs(output_dir, exist_ok=True)

# Guarda el modelo
trainer.save_model(output_dir)

#Carga y prueba del modelo

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model_path = "/content/drive/MyDrive/Colab Notebooks/Tesis Maestria /Modelos Entrenados/modelo_entrenado_T5"
model_T5 = AutoModelForSeq2SeqLM.from_pretrained(model_path)

In [17]:
from transformers import pipeline, T5Tokenizer, T5ForConditionalGeneration

model_checkpoint = "t5-small"

generator = pipeline("text2text-generation", model=model_T5, tokenizer=model_checkpoint)
predictions = generator("Dada una muestra con pH de 5.38., materia orgánica de 1.663102257317878., fósforo de 12.55712389891141., azufre de 2.360335195530726., acidez de 0.5985407250904465., aluminio de 0.477558493382691., calcio de 2.01080385., magnesio de 0.35373262., potasio de 0.109947672., sodio de < 0,10., CICE de 3.173024867090447., y conductividad eléctrica de 0.1553496., ¿cuál es el diagnóstico adecuado?", max_length=300)

In [18]:
print(predictions)

[{'generated_text': 'Suelo con adecuado a adecuado a adecuado a adecuado a adecuado a adecuado a azufre a azufre a recomienda el diagnóstico. Para el diagnóstico adecuado a azufre '}]


In [19]:
# Ejemplo de pregunta de prueba
test_question = "Dada una muestra con pH de 5.38., materia orgánica de 1.663102257317878., fósforo de 12.55712389891141., azufre de 2.360335195530726., acidez de 0.5985407250904465., aluminio de 0.477558493382691., calcio de 2.01080385., magnesio de 0.35373262., potasio de 0.109947672., sodio de < 0,10., CICE de 3.173024867090447., y conductividad eléctrica de 0.1553496., ¿cuál es el diagnóstico adecuado?"

# Generar respuesta
generated_response = generator(test_question, max_length=300)[0]['generated_text']

# Respuesta esperada para comparación (esta debería ser proporcionada por un experto)
expected_response = "Suelo con reacción Fuerte a Extremadamente Ácida , relacionada con la elevada saturación de Aluminio, de acuerdo con esto, se considera pertinente suministrar enmiendas calcáreas, para manejo de Acidez Intercambiable. Porcentaje alto de Materia Orgánica, in dicando adecuada disponibilidad de Nitrógeno, por lo anterior, requiere adiciones moderadas de éste. Los elementos Fósforo y Azufre presentan contenidos bajos, por ende, se aconseja agregarlos al suelo. Los niveles edáficos de Calcio, Magnesio y Potasio so n bajos y medios, por tal motivo, se recomiendan aplicar cada uno de los mismos. La concentración nativa de Zinc es baja, por consiguiente, se sugiere añadirlo al suelo."

# Función para evaluar la similitud semántica
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoModel, AutoTokenizer

semantic_model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
semantic_tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

def get_embedding(text):
    inputs = semantic_tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=128)
    outputs = semantic_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()

generated_embedding = get_embedding(generated_response)
expected_embedding = get_embedding(expected_response)

similarity = cosine_similarity(generated_embedding, expected_embedding)[0][0]
print(f"Similitud semántica: {similarity * 100:.2f}%")


Similitud semántica: 37.21%
